In [1]:
import os
import subprocess
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from Bio import SeqIO
import pprint

### Делаем словарь со списком геномов и словарь белков

In [2]:
genomes_dict = dict()
for genus in os.listdir('data'):
    files = os.listdir('data/'+genus)
    files = list(filter(lambda x: x.endswith('.faa'), files))
    species = list(map(lambda x: x.split(".")[0], files))
    genomes_dict[genus] = species

In [26]:
aa_seqs = dict()
for genus in genomes_dict.keys():
    for genome in genomes_dict[genus]:
        aa_seqs.update(SeqIO.to_dict(SeqIO.parse(f"data/{genus}/{genome}.faa", "fasta")))

In [ ]:
# Список всех геномов
genomes_list = 

## Определяем кластеры

In [14]:
folders = ["data/" + folder + "/*" for folder in os.listdir("data")]
" ".join(folders)

'data/Halomonas/* data/Enterobacter/* data/Mannheimia/* data/Aeromonas/* data/Erwinia/* data/Lysobacter/* data/Klebsiella/* data/Dickeya/* data/Kosakonia/* data/Pseudomonas/*'

In [17]:
subprocess.run("proteinortho5 -project=gammaproteo " + " ".join(folders), shell=True, check=True)

CompletedProcess(args='proteinortho5 -project=gammaproteo data/Halomonas/* data/Enterobacter/* data/Mannheimia/* data/Aeromonas/* data/Erwinia/* data/Lysobacter/* data/Klebsiella/* data/Dickeya/* data/Kosakonia/* data/Pseudomonas/*', returncode=0)

In [18]:
! mv gammaproteo* proteinortho/

## Работаем со статистикой

In [ ]:
clusters = pd.read_csv("proteinortho/gammaproteo.proteinortho", sep='\t')
clusters = clusters.loc[clusters['# Species'] >= 25]
clusters.replace(to_replace='*', value=np.NaN, inplace=True)
clusters

#### Рисуем гистограмму распределения родов, видов и генов в кластерах

In [126]:
! mkdir -p pictures
sns.set_theme()
sns.histplot(data=clusters, x="# Species", discrete=True)
plt.title("Distribution of species in clusters")
plt.tight_layout()
plt.savefig(f"pictures/species_distribution.png", dpi=400)
plt.close()

sns.histplot(data=clusters, x="Genes", discrete=True)
plt.title("Distribution of genes in clusters")
plt.tight_layout()
plt.savefig(f"pictures/genes_distribution.png", dpi=400)
plt.close()

sns.histplot(data=cluster_stat, x="Genuses", discrete=True)
plt.title("Distribution of genuses in clusters")
plt.tight_layout()
plt.savefig(f"pictures/genuses_distribution.png", dpi=400)
plt.close()

In [4]:
'''
aa_seqs = dict()
for genus in genomes_dict.keys():
    temp_dict = dict()
    for genome in genomes_dict[genus]:
        temp_dict[genome] = SeqIO.to_dict(SeqIO.parse(f"data/{genus}/{genome}.faa", "fasta"))
    aa_seqs[genus] = temp_dict
del temp_dict
'''

#### Готовим таблицу

In [50]:
def return_function_by_id(ID):
    Seq = aa_seqs[ID]
    descr = Seq.description
    if ":" in descr:
        return descr[descr.find(":")+1 : descr.find("[")].strip().capitalize()
    else:
        return descr[descr.find(" ")+1 : descr.find("[")].strip().capitalize()

return_function_by_id('BBW73724.1')


'Hypothetical protein thokle011_00140'

In [89]:
def return_function_of_cluster(row):
    row = row[3:53]
    row = row.dropna()
    # To deal with multiple genes in one genome
    row = ",".join(row).split(',')
    list_of_functions = list(map(return_function_by_id, row))
    values, counts = np.unique(list_of_functions, return_counts=True)
    return values[np.argmax(counts)]

return_function_of_cluster(clusters.iloc[8])

'50s ribosomal protein l29'

In [117]:
def number_of_genuses(row):
    row = row[3:53]
    row = row.dropna()
    names = list(map(lambda x: x.split('.')[0], row.index))
    counter = 0
    for key in genomes_dict:
        for name in names:
            if name in genomes_dict[key]:
                counter += 1
                break
    return counter

number_of_genuses(clusters.iloc[10])

8

In [119]:
cluster_stat = pd.DataFrame(clusters[['# Species', 'Genes']])
cluster_stat['Genuses'] = clusters.apply(number_of_genuses, axis=1)
cluster_stat['Function'] = clusters.apply(return_function_of_cluster, axis=1)
cluster_stat

,# Species,Genes,Genuses,Function
1,27,27,10,30s ribosomal protein s19
2,29,29,10,50s ribosomal protein l35
3,34,34,10,50s ribosomal protein l20
4,28,28,10,F0f1 atp synthase subunit c
5,35,35,10,30s ribosomal protein s13
...,...,...,...,...
9869,25,25,6,Multidrug efflux mfs transporter mdth
10205,25,33,8,Replication endonuclease
10210,29,30,10,Superoxide dismutase family protein
10240,25,26,7,M4 family metallopeptidase
